In [ ]:
import  numpy as np
import pandas as pd

In [ ]:
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

In [ ]:
from nltk.util import ngrams
from collections import Counter
from gensim.models import word2vec

In [ ]:
data = pd.read_csv('../input/ziphack/train_file.csv')
test = pd.read_csv('../input/ziphack/test_file.csv')

In [ ]:
data["MaterialType"] = data['MaterialType'].str.replace('[^\w\s]','')

In [ ]:
data.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth','ID'],inplace=True,axis=1)
test.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],inplace=True,axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as skp
label_encoder = LabelEncoder()

In [ ]:
led = LabelEncoder()
led.fit(data["MaterialType"])

data["MaterialType"] = led.transform(data["MaterialType"].values)

In [ ]:
y = data["MaterialType"]

In [ ]:
data = data.apply(lambda x: x.astype(str).str.lower())
test = test.apply(lambda x: x.astype(str).str.lower())

In [ ]:
data["Title"] = data['Title'].str.replace('[^\w\s]','')
test["Title"] = test['Title'].str.replace('[^\w\s]','')
data["Subjects"] = data['Subjects'].str.replace('[^\w\s]','')
test["Subjects"] = test['Subjects'].str.replace('[^\w\s]','')

In [ ]:
data.head()

In [ ]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [ ]:
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

In [ ]:
data['Title'] = data['Title'].apply(stopwords)
data['Subjects'] = data['Subjects'].apply(stopwords)
test['Subjects'] = test['Subjects'].apply(stopwords)

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
def pre(data):
    word = lemmatizer.lemmatize(data) 
    return word

In [ ]:
data['Title'] = data['Title'].apply(pre)
data['Subjects'] = data['Subjects'].apply(pre)
test['Subjects'] = test['Subjects'].apply(pre)
test['Title'] = test['Title'].apply(pre)

In [ ]:
data["Subjects"] = data['Subjects'].replace('nan','')

In [ ]:
data.head()

In [ ]:
data["Phrase"] = data["Title"].map(str) + ' ' + data["Subjects"] 
test["Phrase"] = test["Title"].map(str) + ' ' + test["Subjects"]

In [ ]:
data.head()

In [ ]:
import os
import re

import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.layers import *
from keras.models import Model


# For reproducibility.
seed = 7
np.random.seed(seed)
tf.set_random_seed(seed)
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
data.Phrase = data.Phrase.apply(lambda x: re.sub(r'[0-9]+', '0', x))
test.Phrase = test.Phrase.apply(lambda x: re.sub(r'[0-9]+', '0', x))

x_train = data['Phrase'].values
x_test  = test['Phrase'].values
y_train = data['MaterialType'].values
x = np.r_[x_train, x_test]

In [ ]:
tokenizer = Tokenizer(lower=True, filters='\n\t')
tokenizer.fit_on_texts(x)
x_train = tokenizer.texts_to_sequences(x_train)
x_test  = tokenizer.texts_to_sequences(x_test)
vocab_size = len(tokenizer.word_index) + 1  # +1 is for zero padding.
print('vocabulary size: {}'.format(vocab_size))

In [ ]:
EMBEDDING_FILE =  '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

def load_embeddings(filename):
    embeddings = {}
    with open(filename) as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

embeddings = load_embeddings(EMBEDDING_FILE)

In [ ]:
maxlen = len(max((s for s in np.r_[x_train, x_test]), key=len))
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding='post')
print('maxlen: {}'.format(maxlen))
print(x_train.shape)
print(x_test.shape)

In [ ]:
def filter_embeddings(embeddings, word_index, vocab_size, dim=300):
    embedding_matrix = np.zeros([vocab_size, dim])
    for word, i in word_index.items():
        if i >= vocab_size:
            continue
        vector = embeddings.get(word)
        if vector is not None:
            embedding_matrix[i] = vector
    return embedding_matrix

embedding_size = 300
embedding_matrix = filter_embeddings(embeddings, tokenizer.word_index,
                                     vocab_size, embedding_size)
print('OOV: {}'.format(len(set(tokenizer.word_index) - set(embeddings))))


In [ ]:
class Attention(Layer):
    """
    Keras Layer that implements an Attention mechanism for temporal data.
    Supports Masking.
    Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    :param kwargs:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(Attention())
    """
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
def build_model(maxlen, vocab_size, embedding_size, embedding_matrix):
    input_words = Input((maxlen, ))
    x_words = Embedding(vocab_size,
                        embedding_size,
                        weights=[embedding_matrix],
                        mask_zero=True,
                        trainable=False)(input_words)
    x_words = SpatialDropout1D(0.3)(x_words)
    x_words = Bidirectional(LSTM(50, return_sequences=True))(x_words)
    x = Attention(maxlen)(x_words)
    x = Dropout(0.2)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.2)(x)
    pred = Dense(8, activation='softmax')(x)

    model = Model(inputs=input_words, outputs=pred)
    return model

model = build_model(maxlen, vocab_size, embedding_size, embedding_matrix)
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
save_file = 'model.h5'
history = model.fit(x_train, y_train,
                    epochs=20, verbose=1,
                    batch_size=1024, shuffle=True)

In [ ]:
y_pred = model.predict(x_test, batch_size=1024)
y_pred = y_pred.argmax(axis=1).astype(int)
y_pred.shape

In [ ]:
submission = pd.DataFrame({'ID':test['ID'],'MaterialType': y_pred})

In [ ]:
submission["MaterialType"] = led.inverse_transform(submission["MaterialType"])
submission = submission.apply(lambda x: x.astype(str).str.upper())
submission.to_csv('submission_cnn.csv', index=False)

In [ ]:
submission